In [1]:
from openai import OpenAI
client = OpenAI()

prompt = """
Context information is below.
=============================

Prerequisites!

This tutorial assumes you already have OmniPanel credentials and access to your API key for communicating with Authologic APIs. If you’re not at this stage yet, feel free to reach out to us for help.

TIP

We use the curl tool to show the API operation. If you have not dealt with it, you can find a short guide on it here. Of course, there is nothing to prevent you from using the swagger tool directly, which you will find at the link: here, or any other tool, such as Postman or Insomnia.
The Conversation


For the sake of simplicity, we’ll refer to the process of interacting with the API as a conversation.

Whenever you want to:

verify a user’s identity,
check AML lists, or
initiate a verification process,
you start a new conversation.

Creating a Conversation


In this 5-minute tutorial, we’ll simulate a user identity verification process. We use a simulation to keep things simple and to focus on the core interactions needed to get you started.

We will use the curl tool to start a new conversation - one that will return the user first and last name. Keep in mind that the following example requires you to replace the my_login with the credentials that you can find in the OmniPanel and my_callback_url_here with an address that our system will return the data to.

For my_callback_url_here you could use https://webhook.site/ to generate callback url which will be responsible for receiving user information from Authologic. Whether you will use webhook.site or other similar tool it is crucial that it will correctly receive http request since it is required for the process of identification.

Create Conversation
curl -X POST -u my_login "https://sandbox.authologic.com/api/conversations" \
-H "Accept: application/vnd.authologic.v1.1+json" \
-H "Content-Type: application/vnd.authologic.v1.1+json" \
-d '{
    "userKey": "7dfb9ded-c38f-49ae-95e2-307283a0b1f6",
    "returnUrl": "https://id.sandbox.authologic.com/c/{conversationId}/thankYou",
    "callbackUrl": "my_callback_url_here",
    "strategy": "public:sandbox",
        "query": {
            "identity": {
                "requireOneOf": [
                    [ "PERSON_NAME_FIRSTNAME", "PERSON_NAME_LASTNAME"]
                ]
            }
        }
    }'
Of course, instead of my_login enter your login. You should be prompted for a password - at that point you should enter the API key (do not confuse it with the password to the customer panel).


TIP

The example given is for a test environment. If you perform these operations on the production environment that was provided to you then the public:sandbox strategy is not available. You can omit the strategy field to use the default strategy or use another one that we gave you the name of when determining your needs. As a result, nothing bad will happen, but remember that the production environment requires real data. The collected data will also be real.
The response of the above command should look something like this:

{
    "id":"c12c1adc-3ff0-4d32-b95c-c593135c903e",
    "userKey":"7dfb9ded-c38f-49ae-95e2-307283a0b1f6",
    "url":"https://sandbox.authologic.com/c/c12c1adc-3ff0-4d32-b95c-c593135c903e",
    "status":"CREATED",
    "result":{
        "identity":{
            "status":"IN_PROGRESS",
            "user":{}
        }
    }
}
In the above response we got the following information:

id - the conversation id. We will need it when inquiring about its status.
userKey - ID of the user to be checked, exactly what you entered when creating the conversation.
url - the address to which the user should be redirected in order to verify the identity.
status - conversation status. CREATED marks a new conversation where the user has not yet started the verification process.
result - conversation result. At the moment we do not have any data, so there appears, within the product for which we asked the status: IN_PROGRESS and an empty structure with data about the user (user)
Identity Check


To start the identity check, direct the user (or yourself during testing) to the url provided in the response in the url field. In the sandbox environment, you’ll see a form where you can enter the details to simulate the identity verification process. Once completed, the user will be redirected to the returnUrl you specified earlier - in the conversation creation request.

Getting the Conversation Results


Now let's check the conversation status. You should get request on url that was provided in callbackUrl field during creating conversation, which should look similar to the following JSON:

{
    "id": "f109ad2b-5e3c-4f9d-b2ea-20379e56f101",
    "created": "2020-12-24T16:00:00.930853456Z",
    "target": "CONVERSATION",
    "event": "FINISHED",
    "payload": {
        "conversation": {
            "id":"c12c1adc-3ff0-4d32-b95c-c593135c903e",
            "userKey":"7dfb9ded-c38f-49ae-95e2-307283a0b1f6",
            "url":"https://sandbox.authologic.com/c/c12c1adc-3ff0-4d32-b95c-c593135c903e",
            "status":"FINISHED",
            "result":{
                "identity":{
                    "status":"FINISHED",
                    "errors": [],
                    "user":{
                        "person":{
                            "name":{
                                "firstName":"Jan",
                                "lastName":"Testowy"
                            }
                        }
                    }
                }
            }
        }
    }
}

Congratulation!!

That's it. You managed to use API and get data.
=============================
Given the above context infromation and without any prior knoweldge, answer the query.
Query: Generate Typescript code based on the documentation and the curl examples in it. Provide comments.
"""

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are a system that generates Typescript code with extensive comments based on curl examples."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(completion.choices[0].message.content)

Certainly! Below is a TypeScript code example that demonstrates how to create a new conversation with the Authologic API based on the provided curl example. The code includes extensive comments to explain each part of the process.

```typescript
import axios from 'axios';

// Define the base URL for the Authologic Sandbox API
const BASE_URL = "https://sandbox.authologic.com/api/conversations";

// Configuration for the API request
const config = {
    // This assumes you're using user login as part of Basic Authentication.
    auth: {
        username: 'your_login_here', // Replace this with your OmniPanel login
        password: 'your_api_key_here' // Use your API key instead of a password
    },
    headers: {
        "Accept": "application/vnd.authologic.v1.1+json",
        "Content-Type": "application/vnd.authologic.v1.1+json",
    }
};

// Define the body of the POST request as a JSON object
const requestBody = {
    userKey: "7dfb9ded-c38f-49ae-95e2-307283a0b1f6", // Unique ident